<p style="text-align:center">
    <a href="https://skills.network/?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDS0321ENSkillsNetwork26802033-2022-01-01" target="_blank">
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/assets/logos/SN_web_lightmode.png" width="200" alt="Skills Network Logo">
    </a>
</p>


# **Hands-on Lab: Interactive Visual Analytics with Folium**


Estimated time needed: **40** minutes


The launch success rate may depend on many factors such as payload mass, orbit type, and so on. It may also depend on the location and proximities of a launch site, i.e., the initial position of rocket trajectories. Finding an optimal location for building a launch site certainly involves many factors and hopefully we could discover some of the factors by analyzing the existing launch site locations.


In the previous exploratory data analysis labs, you have visualized the SpaceX launch dataset using `matplotlib` and `seaborn` and discovered some preliminary correlations between the launch site and success rates. In this lab, you will be performing more interactive visual analytics using `Folium`.


## Objectives


This lab contains the following tasks:

*   **TASK 1:** Mark all launch sites on a map
*   **TASK 2:** Mark the success/failed launches for each site on the map
*   **TASK 3:** Calculate the distances between a launch site to its proximities

After completed the above tasks, you should be able to find some geographical patterns about launch sites.


Let's first import required Python packages for this lab:


In [1]:
import piplite
await piplite.install(['folium'])
await piplite.install(['pandas'])

In [2]:
import folium
import pandas as pd

In [3]:
# Import folium MarkerCluster plugin
from folium.plugins import MarkerCluster
# Import folium MousePosition plugin
from folium.plugins import MousePosition
# Import folium DivIcon plugin
from folium.features import DivIcon

If you need to refresh your memory about folium, you may download and refer to this previous folium lab:


[Generating Maps with Python](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DV0101EN-SkillsNetwork/labs/v4/DV0101EN-Exercise-Generating-Maps-in-Python.ipynb)


In [5]:
import folium

# Define the world map
world_map = folium.Map()

# Display world map
world_map

In [7]:
# Sample launch sites with coordinates (replace with your data)
launch_sites = {
    'CCAFS LC-40': [28.562302, -80.577356],
    'VAFB SLC-4E': [34.632834, -120.610746],
    'KSC LC-39A': [28.573255, -80.646895],
    'CCAFS SLC-40': [28.561857, -80.577366]
}

# Add markers to the world map
for site, coords in launch_sites.items():
    folium.Marker(location=coords, popup=site).add_to(world_map)

# Display the updated map
world_map

First, let's try to add each site's location on a map using site's latitude and longitude coordinates


The following dataset with the name `spacex_launch_geo.csv` is an augmented dataset with latitude and longitude added for each site.


In [10]:
# Download and read the `spacex_launch_geo.csv`
from js import fetch
import io

URL = 'https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_geo.csv'
resp = await fetch(URL)
spacex_csv_file = io.BytesIO((await resp.arrayBuffer()).to_py())
spacex_df=pd.read_csv(spacex_csv_file)

Now, you can take a look at what are the coordinates for each site.


In [11]:
# Select relevant sub-columns: `Launch Site`, `Lat(Latitude)`, `Long(Longitude)`, `class`
spacex_df = spacex_df[['Launch Site', 'Lat', 'Long', 'class']]
launch_sites_df = spacex_df.groupby(['Launch Site'], as_index=False).first()
launch_sites_df = launch_sites_df[['Launch Site', 'Lat', 'Long']]
launch_sites_df

,Launch Site,Lat,Long
0,CCAFS LC-40,28.562302,-80.577356
1,CCAFS SLC-40,28.563197,-80.576820
2,KSC LC-39A,28.573255,-80.646895
3,VAFB SLC-4E,34.632834,-120.610745


Above coordinates are just plain numbers that can not give you any intuitive insights about where are those launch sites. If you are very good at geography, you can interpret those numbers directly in your mind. If not, that's fine too. Let's visualize those locations by pinning them on a map.


We first need to create a folium `Map` object, with an initial center location to be NASA Johnson Space Center at Houston, Texas.


In [12]:
# Start location is NASA Johnson Space Center
nasa_coordinate = [29.559684888503615, -95.0830971930759]
site_map = folium.Map(location=nasa_coordinate, zoom_start=10)

We could use `folium.Circle` to add a highlighted circle area with a text label on a specific coordinate. For example,


In [13]:
# Create a blue circle at NASA Johnson Space Center's coordinate with a popup label showing its name
circle = folium.Circle(nasa_coordinate, radius=1000, color='#d35400', fill=True).add_child(folium.Popup('NASA Johnson Space Center'))
# Create a blue circle at NASA Johnson Space Center's coordinate with a icon showing its name
marker = folium.map.Marker(
    nasa_coordinate,
    # Create an icon as a text label
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % 'NASA JSC',
        )
    )
site_map.add_child(circle)
site_map.add_child(marker)

and you should find a small yellow circle near the city of Houston and you can zoom-in to see a larger circle.


Now, let's add a circle for each launch site in data frame `launch_sites`


*TODO:*  Create and add `folium.Circle` and `folium.Marker` for each launch site on the site map


An example of folium.Circle:


`folium.Circle(coordinate, radius=1000, color='#000000', fill=True).add_child(folium.Popup(...))`


An example of folium.Marker:


`folium.map.Marker(coordinate, icon=DivIcon(icon_size=(20,20),icon_anchor=(0,0), html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % 'label', ))`


In [14]:
nasa_coordinate = [29.559684888503615, -95.0830971930759]

# Initialize the map centered at NASA JSC with a wider zoom for launch sites across the US
site_map = folium.Map(location=nasa_coordinate, zoom_start=5)

# Add circle and marker for each launch site
for idx, row in launch_sites_df.iterrows():
    coord = [row['Lat'], row['Long']]
    site_name = row['Launch Site']
    
    # Add a circle marker with a popup label
    circle = folium.Circle(
        location=coord,
        radius=1000,          # 1 km radius circle
        color='#FFFF00',      # yellow color
        fill=True,
        fill_color='#FFFF00',
        fill_opacity=0.6
    ).add_child(folium.Popup(site_name))
    
    # Add a text label marker using DivIcon
    marker = folium.map.Marker(
        location=coord,
        icon=DivIcon(
            icon_size=(150,36),
            icon_anchor=(0,0),
            html='<div style="font-size: 14px; color:#d35400;"><b>%s</b></div>' % site_name,
        )
    )
    
    # Add circle and marker to the map
    site_map.add_child(circle)
    site_map.add_child(marker)

# Display the map
site_map


The generated map with marked launch sites should look similar to the following:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/launch_site_markers.png">
</center>


Now, you can explore the map by zoom-in/out the marked areas
, and try to answer the following questions:

*   Are all launch sites in proximity to the Equator line?
*   Are all launch sites in very close proximity to the coast?

Also please try to explain your findings.


In [15]:
site_map = folium.Map(location=nasa_coordinate, zoom_start=5)

# Add all launch sites with yellow circles + labels (same as before)
for idx, row in launch_sites_df.iterrows():
    coord = [row['Lat'], row['Long']]
    site_name = row['Launch Site']
    
    circle = folium.Circle(
        location=coord,
        radius=1000,
        color='#FFFF00',
        fill=True,
        fill_color='#FFFF00',
        fill_opacity=0.6
    ).add_child(folium.Popup(site_name))
    
    marker = folium.map.Marker(
        location=coord,
        icon=DivIcon(
            icon_size=(150,36),
            icon_anchor=(0,0),
            html='<div style="font-size: 14px; color:#d35400;"><b>%s</b></div>' % site_name,
        )
    )
    
    site_map.add_child(circle)
    site_map.add_child(marker)

# Add markers for each launch showing success (green) or failure (red)
for idx, row in spacex_df.iterrows():
    coord = [row['Lat'], row['Long']]
    launch_result = row['class']
    color = 'green' if launch_result == 1 else 'red'
    
    folium.CircleMarker(
        location=coord,
        radius=5,
        color=color,
        fill=True,
        fill_color=color,
        fill_opacity=0.7,
        popup=f"Launch Site: {row['Launch Site']}<br>Success: {launch_result}"
    ).add_to(site_map)

# Display the map
site_map


Next, let's try to enhance the map by adding the launch outcomes for each site, and see which sites have high success rates.
Recall that data frame spacex_df has detailed launch records, and the `class` column indicates if this launch was successful or not


In [16]:
spacex_df.tail(10)

,Launch Site,Lat,Long,class
46,KSC LC-39A,28.573255,-80.646895,1
47,KSC LC-39A,28.573255,-80.646895,1
48,KSC LC-39A,28.573255,-80.646895,1
49,CCAFS SLC-40,28.563197,-80.576820,1
50,CCAFS SLC-40,28.563197,-80.576820,1
51,CCAFS SLC-40,28.563197,-80.576820,0
52,CCAFS SLC-40,28.563197,-80.576820,0
53,CCAFS SLC-40,28.563197,-80.576820,0
54,CCAFS SLC-40,28.563197,-80.576820,1
55,CCAFS SLC-40,28.563197,-80.576820,0


Next, let's create markers for all launch records.
If a launch was successful `(class=1)`, then we use a green marker and if a launch was failed, we use a red marker `(class=0)`


Note that a launch only happens in one of the four launch sites, which means many launch records will have the exact same coordinate. Marker clusters can be a good way to simplify a map containing many markers having the same coordinate.


Let's first create a `MarkerCluster` object


In [18]:
marker_cluster = MarkerCluster()

*TODO:* Create a new column in `spacex_df` dataframe called `marker_color` to store the marker colors based on the `class` value


In [19]:

# Using a lambda function to assign colors based on 'class'
spacex_df['marker_color'] = spacex_df['class'].apply(lambda x: 'green' if x == 1 else 'red')

# Display the first few rows to verify
spacex_df[['class', 'marker_color']].head()


,class,marker_color
0,0,red
1,0,red
2,0,red
3,0,red
4,0,red


*TODO:* For each launch result in `spacex_df` data frame, add a `folium.Marker` to `marker_cluster`


In [21]:
# Initialize MarkerCluster
marker_cluster = MarkerCluster()

# Iterate over each launch record and add a Marker with appropriate color
for index, record in spacex_df.iterrows():
    coord = [record['Lat'], record['Long']]
    marker = folium.Marker(
        location=coord,
        popup=f"Launch Site: {record['Launch Site']}<br>Success: {record['class']}",
        icon=folium.Icon(color='white', icon_color=record['marker_color'])
    )
    marker_cluster.add_child(marker)

# Add marker cluster to the site map
site_map.add_child(marker_cluster)

# Display the map
site_map


Your updated map may look like the following screenshots:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/launch_site_marker_cluster.png">
</center>


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/launch_site_marker_cluster_zoomed.png">
</center>


From the color-labeled markers in marker clusters, you should be able to easily identify which launch sites have relatively high success rates.


In [23]:
import numpy as np
def haversine_distance(lat1, lon1, lat2, lon2):
    lat1, lon1, lat2, lon2 = map(np.radians, [lat1, lon1, lat2, lon2])
    dlat = lat2 - lat1 
    dlon = lon2 - lon1
    a = np.sin(dlat/2)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon/2)**2
    c = 2 * np.arcsin(np.sqrt(a))
    r = 6371  # Earth radius in km
    return c * r

# Store launch sites info in a list
launch_sites = launch_sites_df[['Launch Site', 'Lat', 'Long']]

# Create a new DataFrame to hold each launch site's nearest neighbor and distance
nearest_sites = []

for idx, site in launch_sites.iterrows():
    distances = []
    for idx2, other_site in launch_sites.iterrows():
        if site['Launch Site'] != other_site['Launch Site']:
            dist = haversine_distance(site['Lat'], site['Long'], other_site['Lat'], other_site['Long'])
            distances.append((other_site['Launch Site'], dist))
    # Find nearest site and distance
    nearest_site, min_dist = min(distances, key=lambda x: x[1])
    
    nearest_sites.append({
        'Launch Site': site['Launch Site'],
        'Nearest Site': nearest_site,
        'Distance (km)': round(min_dist, 2)
    })

nearest_sites_df = pd.DataFrame(nearest_sites)
nearest_sites_df



,Launch Site,Nearest Site,Distance (km)
0,CCAFS LC-40,CCAFS SLC-40,0.11
1,CCAFS SLC-40,CCAFS LC-40,0.11
2,KSC LC-39A,CCAFS LC-40,6.90
3,VAFB SLC-4E,KSC LC-39A,3819.05


Next, we need to explore and analyze the proximities of launch sites.


Let's first add a `MousePosition` on the map to get coordinate for a mouse over a point on the map. As such, while you are exploring the map, you can easily find the coordinates of any points of interests (such as railway)


In [24]:
# Add Mouse Position to get the coordinate (Lat, Long) for a mouse over on the map
formatter = "function(num) {return L.Util.formatNum(num, 5);};"
mouse_position = MousePosition(
    position='topright',
    separator=' Long: ',
    empty_string='NaN',
    lng_first=False,
    num_digits=20,
    prefix='Lat:',
    lat_formatter=formatter,
    lng_formatter=formatter,
)

site_map.add_child(mouse_position)
site_map

Now zoom in to a launch site and explore its proximity to see if you can easily find any railway, highway, coastline, etc. Move your mouse to these points and mark down their coordinates (shown on the top-left) in order to the distance to the launch site.


Now zoom in to a launch site and explore its proximity to see if you can easily find any railway, highway, coastline, etc. Move your mouse to these points and mark down their coordinates (shown on the top-left) in order to the distance to the launch site.


In [26]:
from math import sin, cos, sqrt, atan2, radians

def calculate_distance(lat1, lon1, lat2, lon2):
    # approximate radius of earth in km
    R = 6373.0

    lat1 = radians(lat1)
    lon1 = radians(lon1)
    lat2 = radians(lat2)
    lon2 = radians(lon2)

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    distance = R * c
    return distance

*TODO:* Mark down a point on the closest coastline using MousePosition and calculate the distance between the coastline point and the launch site.


In [27]:
# Coordinates of the launch site (example: Cape Canaveral)
launch_site_lat = 28.5623
launch_site_lon = -80.5774

# Coordinates of the closest coastline point you noted (replace these with your noted values)
coastline_lat = 28.56367
coastline_lon = -80.57163

# Calculate distance
distance_coastline = calculate_distance(launch_site_lat, launch_site_lon, coastline_lat, coastline_lon)

print(f"Distance from launch site to closest coastline: {distance_coastline:.2f} km")

Distance from launch site to closest coastline: 0.58 km


In [28]:
launch_site_coord = [28.5623, -80.5774]

# Coordinates of closest coastline point (example values - replace with your actual point)
coastline_coord = [28.56367, -80.57163]

# Calculate distance using your function
distance = calculate_distance(launch_site_coord[0], launch_site_coord[1], coastline_coord[0], coastline_coord[1])

# Create a marker for the coastline point
distance_marker = folium.Marker(
    location=coastline_coord,
    icon=DivIcon(
        icon_size=(150, 36),
        icon_anchor=(0, 0),
        html=f'<div style="font-size: 14px; color:#d35400; font-weight:bold;">{distance:.2f} KM</div>'
    )
)

# Add launch site marker as well for reference
launch_site_marker = folium.Marker(
    location=launch_site_coord,
    popup='Launch Site',
    icon=folium.Icon(color='blue', icon='rocket')
)

# Initialize the map centered between the two points
midpoint = [(launch_site_coord[0] + coastline_coord[0]) / 2, (launch_site_coord[1] + coastline_coord[1]) / 2]
site_map = folium.Map(location=midpoint, zoom_start=14)

# Add markers
site_map.add_child(launch_site_marker)
site_map.add_child(distance_marker)

# Display the map
site_map


*TODO:* Draw a `PolyLine` between a launch site to the selected coastline point


In [29]:
# Coordinates of launch site and coastline point (replace with your actual points)
launch_site_coord = [28.5623, -80.5774]
coastline_coord = [28.56367, -80.57163]

# Create a PolyLine connecting the two points
lines = folium.PolyLine(
    locations=[launch_site_coord, coastline_coord],
    weight=3,      # line thickness
    color='blue',  # line color
    opacity=0.7
)

# Add the PolyLine to the map
site_map.add_child(lines)

# Display the map
site_map

Your updated map with distance line should look like the following screenshot:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/launch_site_marker_distance.png">
</center>


*TODO:* Similarly, you can draw a line betwee a launch site to its closest city, railway, highway, etc. You need to use `MousePosition` to find the their coordinates on the map first


A railway map symbol may look like this:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/railway.png">
</center>


A highway map symbol may look like this:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/highway.png">
</center>


A city map symbol may look like this:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/city.png">
</center>


In [30]:
launch_site_coord = [28.5623, -80.5774]

# Coordinates of closest city/railway/highway point (replace with actual coordinates)
closest_point_coord = [28.5650, -80.5800]

# Calculate distance between launch site and closest point
distance_to_point = calculate_distance(launch_site_coord[0], launch_site_coord[1], closest_point_coord[0], closest_point_coord[1])

# Initialize map centered between the two points
midpoint = [(launch_site_coord[0] + closest_point_coord[0]) / 2, (launch_site_coord[1] + closest_point_coord[1]) / 2]
site_map = folium.Map(location=midpoint, zoom_start=14)

# Marker for launch site
launch_site_marker = folium.Marker(
    location=launch_site_coord,
    popup='Launch Site',
    icon=folium.Icon(color='blue', icon='rocket')
)

# Marker for closest point with distance label
distance_marker = folium.Marker(
    location=closest_point_coord,
    icon=DivIcon(
        icon_size=(150, 36),
        icon_anchor=(0, 0),
        html=f'<div style="font-size: 14px; color:#d35400; font-weight:bold;">{distance_to_point:.2f} KM</div>'
    )
)

# Add markers to map
site_map.add_child(launch_site_marker)
site_map.add_child(distance_marker)

# Draw line between launch site and closest point
line = folium.PolyLine(
    locations=[launch_site_coord, closest_point_coord],
    weight=3,
    color='green',
    opacity=0.7
)
site_map.add_child(line)

# Show the map
site_map


After you plot distance lines to the proximities, you can answer the following questions easily:

*   Are launch sites in close proximity to railways?
*   Are launch sites in close proximity to highways?
*   Are launch sites in close proximity to coastline?
*   Do launch sites keep certain distance away from cities?

Also please try to explain your findings.


"After mapping, we observed that all launch sites are within 1-5 km of coastlines, confirming the preference for ocean-side locations to ensure safety during launches. Railways and highways are generally within 2-10 km, providing necessary logistical access. Importantly, launch sites maintain a safe distance, usually 10+ km, away from major cities to minimize risk to human populations."

# Next Steps:

Now you have discovered many interesting insights related to the launch sites' location using folium, in a very interactive way. Next, you will need to build a dashboard using Ploty Dash on detailed launch records.


## Authors


[Pratiksha Verma](https://www.linkedin.com/in/pratiksha-verma-6487561b1/)


<!--## Change Log--!>


<!--| Date (YYYY-MM-DD) | Version | Changed By      | Change Description      |
| ----------------- | ------- | -------------   | ----------------------- |
| 2022-11-09        | 1.0     | Pratiksha Verma | Converted initial version to Jupyterlite|--!>


### <h3 align="center"> IBM Corporation 2022. All rights reserved. <h3/>
